# # 모듈 및 함수
---

In [2]:
# from proj_2d_box import PointHandler
import json
from scipy.spatial.transform import Rotation as R
import numpy as np
import os
import shutil
import glob
import re
import open3d as o3d
import math
import copy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
class Fit3dBox:

    def __init__(self, bf_label, tf_label, tf_pcd, weather):
        self.bf_anns = bf_label['annotation']
        self.tf_anns = tf_label['annotation']
        self.tf_pcd = tf_pcd
        self.weather = weather

        # Best Frame의 객체 id와 box index를 매칭
        bf_id_idx = {}
        for i, bf_ann in enumerate(self.bf_anns):
            bf_id_idx[bf_ann['id']] = i
        self.bf_id_idx = bf_id_idx

    
    def subbox_select(self, subbox_list, category):
        if category == 'OVERPASS' or category == 'TUNNEL':
            y_loc = {}
            for i, subbox in enumerate(subbox_list):
                y_loc[i] = subbox['location'][1]
            idx_yloc = sorted(y_loc.items(), key=lambda x: x[1])

            left_box = subbox_list[idx_yloc[0][0]]
            center_box = subbox_list[idx_yloc[1][0]]
            right_box = subbox_list[idx_yloc[2][0]]
            
            return left_box, center_box, right_box


    def find_floor(self, pcd_in_dim):
        x_cord = np.around(pcd_in_dim[:, 0])
        pcd_in_dim[:, 0] = x_cord

        floor = []
        for x_dot in set(x_cord):
            z_dot = pcd_in_dim[pcd_in_dim[:, 0]==x_dot, 2]
            floor.append(np.min(z_dot))

        # 배열의 기울기 계산
        gradients = np.diff(floor)

        # 기울기 값이 크게 변하는 구간 제외
        outliers_idx = np.where(abs(gradients) > 1)[0]

        del_idx = []
        for i in outliers_idx:
            if floor[i] >= floor[i+1]:
                del_idx.append(i)
            else:
                del_idx.append(i+1)
        
        floor = np.delete(floor, list(set(del_idx)))
        floor_point = np.max(floor) + 0.01
        
        return floor_point


    def road_sign(self):
        bf_anns = self.bf_anns
        tf_anns = self.tf_anns
        tf_pcd = self.tf_pcd
        bf_id_idx = self.bf_id_idx
        weather = self.weather

        for tf_ann in tf_anns:
            if tf_ann['category']=='ROAD_SIGN' and tf_ann['id'] in bf_id_idx.keys():

                matching_id = bf_id_idx[tf_ann['id']]

                bf_box = bf_anns[matching_id]['3d_box'][0]
                tf_box = tf_ann['3d_box'][0]

                tf_box['dimension'] = bf_box['dimension']

                location = tf_box['location']
                dimension = tf_box['dimension'] # w, h, l
                rotation_y = tf_box['rotation_y']

                extra_range = [0, 1, 1] # w, h, l에 더할 값
                extra_dim = np.array(dimension) + np.array(extra_range)

                rmat, rmat_inv = self._rmat_and_inv(rotation_y)
                
                corners_3d = self._get_3d_corners(location, extra_dim, rmat)
                pcd_in_dim = self._get_pcd_in_dim(tf_pcd, corners_3d, rmat_inv)

                # 원점에서 가장 가까운 x값에 근접하도록 박스를 이동
                x_dim_min = location[0] - extra_dim[2]/2
                x_pcd_min = np.min(pcd_in_dim[:,0])
                x_diff = x_dim_min - x_pcd_min
                    
                if x_diff >= 0: # pcd가 dimension 안에 속한 경우
                    tf_box['location'][0] = location[0] - extra_range[2]/2 + x_diff - 0.01 # extra_range 영향 제거 / 선과 pcd가 닿지 않도록 해주는 여유값 추가 (0.01m)
                else: # pcd가 dimension 밖에 있는 경우
                    tf_box['location'][0] = location[0] - extra_range[2]/2 - x_diff - 0.01

                # 원점에서 가장 가까운 z값에 근접하도록 박스를 이동, weather에 따라 다르게 적용
                if 'Rain' not in weather:
                    z_dim_min = location[2] - extra_dim[1]/2
                    z_pcd_min = np.min(pcd_in_dim[:,2])
                    z_diff = z_dim_min - z_pcd_min

                    if z_diff >= 0: # pcd가 dimension 안에 속한 경우
                        tf_box['location'][2] = location[2] - extra_range[1]/2 + z_diff - 0.01
                    else: # pcd가 dimension 밖에 있는 경우
                        tf_box['location'][2] = location[2] - extra_range[1]/2 - z_diff - 0.01

        return tf_anns

    
    def tunnel(bf_label, tf_label, tf_pcd, weather):
        bf_anns = bf_label['annotation']
        tf_anns = tf_label['annotation']
        
        bf_id_idx = {}
        for i, bf_ann in enumerate(bf_anns):
            bf_id_idx[bf_ann['id']] = i

        def find_edge(location, pcd_in_dim, floor_point, pillar_loc: str='left'):
            margin = 0.25
            temp_pcd_in_dim = copy.deepcopy(pcd_in_dim)
            temp_pcd_in_dim = temp_pcd_in_dim[(temp_pcd_in_dim[:, 2] > floor_point) & \
                                            (temp_pcd_in_dim[:, 0] >= location[0] - margin) & (temp_pcd_in_dim[:, 0] <= location[0] + margin)]
            # front_box = temp_pcd_in_dim[:, [1, 2]]
            z_cord = np.around(temp_pcd_in_dim[:, 2], 1)
            temp_pcd_in_dim[:, 2] = z_cord

            left_edge = []
            right_edge = []

            # 연석 찾기
            for z_dot in sorted(set(z_cord))[:10]:
                y_dot = temp_pcd_in_dim[temp_pcd_in_dim[:, 2]==z_dot, 1]
                if pillar_loc == 'left':
                    right_edge.append(np.min(y_dot))
                elif pillar_loc == 'right':
                    left_edge.append(np.max(y_dot))
                    
            # 터널 벽면 찾기
            for z_dot in sorted(set(z_cord))[10:20]:
                y_dot = temp_pcd_in_dim[temp_pcd_in_dim[:, 2]==z_dot, 1]
                if pillar_loc == 'left':
                    left_edge.append(np.min(y_dot))
                elif pillar_loc == 'right':
                    right_edge.append(np.max(y_dot))
            
            left_point = np.min(left_edge)
            right_point = np.min(right_edge)

            # 터널 높이 정하기
            if pillar_loc == 'left':
                top_edge = pcd_in_dim[(pcd_in_dim[:, 1]>=right_point-margin) & (pcd_in_dim[:, 1]<=right_point+margin) & \
                                    (pcd_in_dim[:, 0]>=location[0]-margin) & (pcd_in_dim[:, 0]<=location[0]+margin) & \
                                    (pcd_in_dim[:, 2]>location[2]), 2]
                top_point = np.min(top_edge)
            elif pillar_loc == 'right':
                top_edge = pcd_in_dim[(pcd_in_dim[:, 1]>=left_point-margin) & (pcd_in_dim[:, 1]<=left_point+margin) & \
                                    (pcd_in_dim[:, 0]>=location[0]-margin) & (pcd_in_dim[:, 0]<=location[0]+margin) & \
                                    (pcd_in_dim[:, 2]>location[2]), 2]
                top_point = np.min(top_edge)

            left_point += margin
            right_point -= margin
            top_point += margin

            return left_point, right_point, top_point

        for tf_ann in tf_anns:
            if tf_ann['category']=='TUNNEL' and tf_ann['id'] in bf_id_idx.keys():

                matching_id = bf_id_idx[tf_ann['id']]

                bf_box = bf_anns[matching_id]['3d_box']
                tf_box = tf_ann['3d_box']

                # 분리라벨링 개수가 동일해야함
                if len(bf_box) == len(tf_box) == 3:

                    left_bf_box, center_bf_box, right_bf_box = subbox_select(bf_box, 'TUNNEL')
                    left_tf_box, center_tf_box, right_tf_box = subbox_select(tf_box, 'TUNNEL')

                    ## 터널 양쪽 기둥
                    pillar_top_points = []
                    for pillar_loc, pillar_tf_box in zip(['left', 'right'], [left_tf_box, right_tf_box]):
                        location = pillar_tf_box['location']
                        dimension = pillar_tf_box['dimension'] # w, h, l
                        rotation_y = pillar_tf_box['rotation_y']

                        extra_range = [1, 2, 0] # w, h, l에 더할 값
                        extra_dim = np.array(dimension) + np.array(extra_range)

                        rmat, rmat_inv = PointHandler()._rmat_and_inv(rotation_y)

                        corners_3d = PointHandler()._get_3d_corners(location, extra_dim, rmat)
                        pcd_in_dim = PointHandler()._get_pcd_in_dim(tf_pcd, corners_3d, rmat_inv)
                        
                        # 지면 찾기
                        floor_point = find_floor(pcd_in_dim)

                        # 기둥 범위 정하기
                        left_point, right_point, top_point = find_edge(location, pcd_in_dim, floor_point, pillar_loc=pillar_loc)

                        w = abs(left_point - right_point)
                        h = abs(top_point - floor_point)

                        y = left_point - w/2
                        z = top_point - h/2

                        pillar_tf_box['location'] = [location[0], y, z]
                        pillar_tf_box['dimension'] = [w, h, dimension[2]]

                        top_points.append(top_point)

                    # 기둥 높이 동일하게 맞추기
                    if pillar_top_points[0] > pillar_top_points[1]: # 왼쪽 기둥이 더 높은 경우
                        pillar_top_point = np.max(pillar_top_points)
                        pillar_bottom_point = right_tf_box['location'][1] - right_tf_box['dimension'][1]/2
                        
                        h = pillar_top_point - pillar_bottom_point
                        right_tf_box['dimension'][1] = h
                        right_tf_box['location'][1] = pillar_bottom_point + h/2

                    elif pillar_top_points[0] < pillar_top_points[1]: # 오른쪽 기둥이 더 높은 경우
                        pillar_top_point = np.max(pillar_top_points)
                        pillar_bottom_point = left_tf_box['location'][1] - left_tf_box['dimension'][1]/2

                        h = pillar_top_point - pillar_bottom_point
                        left_tf_box['dimension'][1] = h
                        left_tf_box['location'][1] = pillar_bottom_point + h/2


                    ## 터널 상단
                    w_max = left_tf_box['location'][1] + left_tf_box['dimension'][0]/2
                    w_min = right_tf_box['location'][1] - right_tf_box['dimension'][0]/2
                    h_max = center_tf_box['location'][2] + center_tf_box['dimension'][1]/2
                    h_min = top_point

                    w = w_max - w_min
                    h = h_max - h_min

                    x = center_tf_box['location'][0]
                    y = w_min + w/2
                    z = h_min + h/2

                    center_tf_box['location'] = [x, y, z]
                    center_tf_box['dimension'] = [w, h, center_tf_box['dimension'][2]]

        return tf_anns



class PointHandler:

    def _rmat_and_inv(self, rotation_y):
        euler_angle = [0,0, rotation_y]
        rot = R.from_euler('xyz', euler_angle, degrees=True)  # type: ignore
        rot_inv = rot.inv()
        rmat = np.array(rot.as_matrix())
        rmat_inv = np.array(rot_inv.as_matrix())

        return rmat, rmat_inv
    

    def _get_3d_corners(self, location, dimension, rmat):
        '''location: [x, y, z]
           dimension: [w, h, l]
           rmat: rotation matrix
           extra_range: [l, w, h]에 추가로 더할 값'''

        x = location[0]
        y = location[1]
        z = location[2]
        l = dimension[2]
        w = dimension[0]
        h = dimension[1]

        x_corners = [l/2, l/2, -l/2, -l/2, l/2, l/2, -l/2, -l/2]
        y_corners = [w/2, -w/2, -w/2, w/2, w/2, -w/2, -w/2, w/2]
        z_corners = [h/2, h/2, h/2, h/2, -h/2, -h/2, -h/2, -h/2]
        corners_3d = np.dot(rmat, np.vstack([x_corners, y_corners, z_corners]))
        corners_3d[0, :] = corners_3d[0, :] + x
        corners_3d[1, :] = corners_3d[1, :] + y
        corners_3d[2, :] = corners_3d[2, :] + z

        return corners_3d


    def _get_pcd_in_dim(self, tf_pcd, corners_3d, rmat_inv):
        x_min = np.min(corners_3d[0, :])
        x_max = np.max(corners_3d[0, :])
        y_min = np.min(corners_3d[1, :])
        y_max = np.max(corners_3d[1, :])
        z_min = np.min(corners_3d[2, :])
        z_max = np.max(corners_3d[2, :])

        pcd_range = tf_pcd[(tf_pcd[:,0] > x_min) & (tf_pcd[:,0] < x_max) & \
                            (tf_pcd[:,1] > y_min) & (tf_pcd[:,1] < y_max) & \
                            (tf_pcd[:,2] > z_min) & (tf_pcd[:,2] < z_max)]

        pcd_in_dim = np.dot(rmat_inv, pcd_range.T)

        return pcd_in_dim.T



class ConvertBox(PointHandler, Fit3dBox):

    def __init__(self, scene_path, bf_num, tf_num, categories):
        '''scene_path: pcd가 저장된 경로
            bf_num: best frame 번호
            tf_num: target frame 번호
            categories: 변환할 카테고리 리스트'''

        self.scene_path = scene_path
        self.lidar_path = os.path.join(self.scene_path, 'Lidar/*.pcd')
        # self.calib_path = os.path.join(self.scene_path, 'calib/Lidar_camera_calib/*.txt')
        self.label_path = os.path.join(self.scene_path, 'result/*.json').replace('source', 'label')
        self.meta_path = os.path.join(self.scene_path, 'Meta/*.json')
        
        self.bf_num = bf_num - 1
        self.tf_num = tf_num - 1

        self.categories = categories

        self.tf_pcd = self.get_tf_pcd()
        self.bf_label, self.tf_label, self.weather = self.get_label()
        # bf_anns = self.bf_label['annotation']
        # tf_anns = self.tf_label['annotation']
        super().__init__(self.bf_label, self.tf_label, self.tf_pcd, self.weather)


    def get_tf_pcd(self):
        tf_pcd_path = sorted(glob.glob(self.lidar_path))[self.tf_num]
        
        pcd = o3d.io.read_point_cloud(tf_pcd_path)
        pcd = np.asarray(pcd.points)
        pcd = np.delete(pcd, np.where((pcd[:,0]<1) | (pcd[:,0] > 80)), 0)

        return pcd


    def get_label(self):
        bf_label_path = sorted(glob.glob(self.label_path))[self.bf_num]
        tf_label_path = sorted(glob.glob(self.label_path))[self.tf_num]
        meta_path = sorted(glob.glob(self.meta_path))[0]

        with open(bf_label_path, 'r') as f:
            bf_label = json.load(f)
        with open(tf_label_path, 'r') as f:
            tf_label = json.load(f)
        with open(meta_path, 'r') as f:
            meta = json.load(f)

        weather = meta['weather']

        # bf_anns = bf_label['annotation']
        # tf_anns = tf_label['annotation']

        return bf_label, tf_label, weather


    def converting(self):

        if 'ROAD_SIGN' in self.categories:
            new_tf_anns = self.road_sign()
            self.tf_label['annotation'] = new_tf_anns

        self.save_new_label()


    def save_new_label(self):
        # 수정한 라벨 저장 위치
        new_label_path = os.path.join(self.scene_path, 'result/').replace('source', 'new_label')
        os.makedirs(os.path.dirname(new_label_path), exist_ok=True)

        # 베스트 프레임 라벨 복사
        bf_label_path = sorted(glob.glob(self.label_path))[self.bf_num]
        new_bf_label_path = bf_label_path.replace('label', 'new_label')
        shutil.copy(bf_label_path, new_bf_label_path)

        # 수정한 타겟 프레임 라벨 저장
        tf_label_path = sorted(glob.glob(self.label_path))[self.tf_num]
        new_tf_label_path = tf_label_path.replace('label', 'new_label')
        with open(new_tf_label_path, 'w') as f:
            json.dump(self.tf_label, f, indent=4)

# # 카테고리
---

## # ROAD_SIGN
---
1. BF추출(w는 유지)
2. BF dim(h, l) 적용
3. tf loc : z는 하단, x는 가장 가까운 지점 : 하단 고려 가능한지 검토 → 육안
4. 비가 올 경우 라이다 확산현상이 있어 z축은 이동하지 않음

In [ ]:
scene_path = '/data/kimgh/NIA48_Algorithm/sampledata/source/normal/17/A_Clip_03169_17'
bf_num = 30
# tf_num = 1
frames = sorted(os.listdir(os.path.join(scene_path, 'Lidar')))
frames = np.arange(1, len(frames)+1, 1)
tf_nums = frames[frames!=bf_num]

categories = ['ROAD_SIGN']

for tf_num in tf_nums:
    ConvertBox(scene_path, bf_num, tf_num, categories).converting()

## # TUNNEL
---
1. 좌우 기둥의 지면 서칭
2. 서칭된 지면에서, 연석의 y edge 지점 서칭 → 기둥 너비 확정
3. 좌우 기둥의 높이 확정
4. 상판 너비 조정
5. 상판 높이 조정
    1) 중심점 기준 1m 내 곡면 존재 확인 → h 조정
    2) 상판이 기둥을 감싸도록 w, l 조정

In [79]:
scene_path = '/data/kimgh/NIA48_Algorithm/sampledata/source/normal/13/S_Clip_08027_13'
bf_num = 30

frames = sorted(os.listdir(os.path.join(scene_path, 'Lidar')))
frames = np.arange(1, len(frames)+1, 1)
tf_nums = frames[frames!=bf_num]

categories = ['TUNNEL']

for tf_num in tf_nums:
    ConvertBox(scene_path, bf_num, tf_num, categories).converting()

In [72]:
scene_path = '/data/kimgh/NIA48_Algorithm/sampledata/source/normal/13/S_Clip_08027_13'

bf_num = 30
tf_num = 1

categories = ['TUNNEL']

convert = ConvertBox(scene_path, bf_num, tf_num, categories)
tf_pcd = convert.get_tf_pcd()
bf_label, tf_label, weather = convert.get_label()

# tf_anns = tunnel(bf_label, tf_label, tf_pcd, weather)
# left_point, right_point, top_point = tunnel(bf_label, tf_label, tf_pcd, weather)
pcd_in_dim, rmat, location = tunnel(bf_label, tf_label, tf_pcd, weather)

In [73]:
def find_edge(location, pcd_in_dim, floor_point, pillar_loc: str='left'):
    margin = 0.25
    temp_pcd_in_dim = copy.deepcopy(pcd_in_dim)
    temp_pcd_in_dim = temp_pcd_in_dim[(temp_pcd_in_dim[:, 2] > floor_point) & \
                                      (temp_pcd_in_dim[:, 0] >= location[0] - margin) & (temp_pcd_in_dim[:, 0] <= location[0] + margin)]
    # front_box = temp_pcd_in_dim[:, [1, 2]]
    z_cord = np.around(temp_pcd_in_dim[:, 2], 1)
    temp_pcd_in_dim[:, 2] = z_cord

    left_edge = []
    right_edge = []

    # 연석 찾기
    for z_dot in sorted(set(z_cord))[:10]:
        y_dot = temp_pcd_in_dim[temp_pcd_in_dim[:, 2]==z_dot, 1]
        if pillar_loc == 'left':
            right_edge.append(np.min(y_dot))
        elif pillar_loc == 'right':
            left_edge.append(np.max(y_dot))
            
    # 터널 벽면 찾기
    for z_dot in sorted(set(z_cord))[10:20]:
        y_dot = temp_pcd_in_dim[temp_pcd_in_dim[:, 2]==z_dot, 1]
        if pillar_loc == 'left':
            left_edge.append(np.min(y_dot))
        elif pillar_loc == 'right':
            right_edge.append(np.max(y_dot))
    
    left_point = np.min(left_edge)
    right_point = np.min(right_edge)

    # 터널 높이 정하기
    if pillar_loc == 'left':
        top_edge = pcd_in_dim[(pcd_in_dim[:, 1]>=right_point-margin) & (pcd_in_dim[:, 1]<=right_point+margin) & \
                              (pcd_in_dim[:, 0]>=location[0]-margin) & (pcd_in_dim[:, 0]<=location[0]+margin) & \
                              (pcd_in_dim[:, 2]>location[2]), 2]
        top_point = np.min(top_edge)
    elif pillar_loc == 'right':
        top_edge = pcd_in_dim[(pcd_in_dim[:, 1]>=left_point-margin) & (pcd_in_dim[:, 1]<=left_point+margin) & \
                              (pcd_in_dim[:, 0]>=location[0]-margin) & (pcd_in_dim[:, 0]<=location[0]+margin) & \
                              (pcd_in_dim[:, 2]>location[2]), 2]
        top_point = np.min(top_edge)

    left_point += margin
    right_point -= margin
    top_point += margin

    return left_point, right_point, top_point

In [78]:
def find_floor(pcd_in_dim):
    temp_pcd_in_dim = copy.deepcopy(pcd_in_dim)
    x_cord = np.around(temp_pcd_in_dim[:, 0])
    temp_pcd_in_dim[:, 0] = x_cord

    floor = []
    for x_dot in set(x_cord):
        z_dot = temp_pcd_in_dim[temp_pcd_in_dim[:, 0]==x_dot, 2]
        floor.append(np.min(z_dot))

    # 배열의 기울기 계산
    gradients = np.diff(floor)

    # 기울기 값이 크게 변하는 구간 제외
    outliers_idx = np.where(abs(gradients) > 1)[0]

    del_idx = []
    for i in outliers_idx:
        if floor[i] >= floor[i+1]:
            del_idx.append(i)
        else:
            del_idx.append(i+1)
    
    floor = np.delete(floor, list(set(del_idx)))
    floor_point = np.max(floor) + 0.01
    
    
    return floor_point


def subbox_select(subbox_list, category):
    if category == 'OVERPASS' or category == 'TUNNEL':
        y_loc = {}
        for i, subbox in enumerate(subbox_list):
            y_loc[i] = subbox['location'][1]
        idx_yloc = sorted(y_loc.items(), key=lambda x: x[1], reverse=True)

        left_box = subbox_list[idx_yloc[0][0]]
        center_box = subbox_list[idx_yloc[1][0]]
        right_box = subbox_list[idx_yloc[2][0]]
        
        return left_box, center_box, right_box


def tunnel(bf_label, tf_label, tf_pcd, weather):
    bf_anns = bf_label['annotation']
    tf_anns = tf_label['annotation']
    
    bf_id_idx = {}
    for i, bf_ann in enumerate(bf_anns):
        bf_id_idx[bf_ann['id']] = i

    def find_edge(location, pcd_in_dim, floor_point, pillar_loc: str='left'):
        margin = 0.25
        temp_pcd_in_dim = copy.deepcopy(pcd_in_dim)
        temp_pcd_in_dim = temp_pcd_in_dim[(temp_pcd_in_dim[:, 2] > floor_point) & \
                                        (temp_pcd_in_dim[:, 0] >= location[0] - margin) & (temp_pcd_in_dim[:, 0] <= location[0] + margin)]
        # front_box = temp_pcd_in_dim[:, [1, 2]]
        z_cord = np.around(temp_pcd_in_dim[:, 2], 1)
        temp_pcd_in_dim[:, 2] = z_cord

        left_edge = []
        right_edge = []

        # 연석 찾기
        for z_dot in sorted(set(z_cord))[:10]:
            y_dot = temp_pcd_in_dim[temp_pcd_in_dim[:, 2]==z_dot, 1]
            if pillar_loc == 'left':
                right_edge.append(np.min(y_dot))
            elif pillar_loc == 'right':
                left_edge.append(np.max(y_dot))
                
        # 터널 벽면 찾기
        for z_dot in sorted(set(z_cord))[10:20]:
            y_dot = temp_pcd_in_dim[temp_pcd_in_dim[:, 2]==z_dot, 1]
            if pillar_loc == 'left':
                left_edge.append(np.min(y_dot))
            elif pillar_loc == 'right':
                right_edge.append(np.max(y_dot))
        
        left_point = np.min(left_edge)
        right_point = np.min(right_edge)

        # 터널 높이 정하기
        if pillar_loc == 'left':
            top_edge = pcd_in_dim[(pcd_in_dim[:, 1]>=right_point-margin) & (pcd_in_dim[:, 1]<=right_point+margin) & \
                                (pcd_in_dim[:, 0]>=location[0]-margin) & (pcd_in_dim[:, 0]<=location[0]+margin) & \
                                (pcd_in_dim[:, 2]>location[2]), 2]
            top_point = np.min(top_edge)
        elif pillar_loc == 'right':
            top_edge = pcd_in_dim[(pcd_in_dim[:, 1]>=left_point-margin) & (pcd_in_dim[:, 1]<=left_point+margin) & \
                                (pcd_in_dim[:, 0]>=location[0]-margin) & (pcd_in_dim[:, 0]<=location[0]+margin) & \
                                (pcd_in_dim[:, 2]>location[2]), 2]
            top_point = np.min(top_edge)

        left_point += margin
        right_point -= margin
        top_point += margin

        return left_point, right_point, top_point

    for tf_ann in tf_anns:
        if tf_ann['category']=='TUNNEL' and tf_ann['id'] in bf_id_idx.keys():

            matching_id = bf_id_idx[tf_ann['id']]

            bf_box = bf_anns[matching_id]['3d_box']
            tf_box = tf_ann['3d_box']

            # 분리라벨링 개수가 동일해야함
            if len(bf_box) == len(tf_box) == 3:

                left_bf_box, center_bf_box, right_bf_box = subbox_select(bf_box, 'TUNNEL')
                left_tf_box, center_tf_box, right_tf_box = subbox_select(tf_box, 'TUNNEL')

                ## 터널 양쪽 기둥
                pillar_top_points = []
                for pillar_loc, pillar_tf_box in zip(['left', 'right'], [left_tf_box, right_tf_box]):
                    location = pillar_tf_box['location']
                    dimension = pillar_tf_box['dimension'] # w, h, l
                    rotation_y = pillar_tf_box['rotation_y']

                    extra_range = [1, 2, 0] # w, h, l에 더할 값
                    extra_dim = np.array(dimension) + np.array(extra_range)

                    rmat, rmat_inv = PointHandler()._rmat_and_inv(rotation_y)

                    corners_3d = PointHandler()._get_3d_corners(location, extra_dim, rmat)
                    pcd_in_dim = PointHandler()._get_pcd_in_dim(tf_pcd, corners_3d, rmat_inv)
                    
                    # 지면 찾기
                    floor_point = find_floor(pcd_in_dim)

                    # 기둥 범위 정하기
                    left_point, right_point, top_point = find_edge(location, pcd_in_dim, floor_point, pillar_loc=pillar_loc)

                    w = abs(left_point - right_point)
                    h = abs(top_point - floor_point)

                    y = left_point - w/2
                    z = top_point - h/2

                    pillar_tf_box['location'] = [location[0], y, z]
                    pillar_tf_box['dimension'] = [w, h, dimension[2]]

                    top_points.append(top_point)

                # 기둥 높이 동일하게 맞추기
                if pillar_top_points[0] > pillar_top_points[1]: # 왼쪽 기둥이 더 높은 경우
                    pillar_top_point = np.max(pillar_top_points)
                    pillar_bottom_point = right_tf_box['location'][1] - right_tf_box['dimension'][1]/2
                    
                    h = pillar_top_point - pillar_bottom_point
                    right_tf_box['dimension'][1] = h
                    right_tf_box['location'][1] = pillar_bottom_point + h/2

                elif pillar_top_points[0] < pillar_top_points[1]: # 오른쪽 기둥이 더 높은 경우
                    pillar_top_point = np.max(pillar_top_points)
                    pillar_bottom_point = left_tf_box['location'][1] - left_tf_box['dimension'][1]/2

                    h = pillar_top_point - pillar_bottom_point
                    left_tf_box['dimension'][1] = h
                    left_tf_box['location'][1] = pillar_bottom_point + h/2


                ## 터널 상단
                w_max = left_tf_box['location'][1] + left_tf_box['dimension'][0]/2
                w_min = right_tf_box['location'][1] - right_tf_box['dimension'][0]/2
                h_max = center_tf_box['location'][2] + center_tf_box['dimension'][1]/2
                h_min = top_point

                w = w_max - w_min
                h = h_max - h_min

                x = center_tf_box['location'][0]
                y = w_min + w/2
                z = h_min + h/2

                center_tf_box['location'] = [x, y, z]
                center_tf_box['dimension'] = [w, h, center_tf_box['dimension'][2]]

    return tf_anns